In [3]:
from neon.models import Model
from neon.data import ArrayIterator
from neon.backends import gen_backend
from hsa import emu_connect
import importlib
import socket
import pandas
import numpy as np
import random

In [ ]:
# hack to import modules in pycharm and browser
import sys
sys.path.append("../")

In [4]:
gen_backend(backend='cpu', batch_size=128)

In [53]:
prim_soc = socket.create_connection(("localhost", 9090))
prim_soc

<socket.socket fd=43, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 33646), raddr=('127.0.0.1', 9090)>

In [54]:
importlib.reload(emu_connect)
emu = emu_connect.Emu2(prim_soc)

In [1]:
emu.close()

NameError: name 'emu' is not defined

In [21]:
nn = Model("mimik2_mario_1_1_first.prm")

In [7]:
emu.load_slot(10)
emu.step()
example_ram = emu.get_ram()

In [18]:
np.frombuffer(example_ram,dtype=np.uint8).reshape((1,2048))

(1, 2048)

In [50]:
gen_backend(backend='cpu', batch_size=1)

In [19]:
example_ram_Neon= ArrayIterator(np.frombuffer(example_ram,dtype=np.uint8).reshape((1,2048)),make_onehot=False)

In [22]:
example_input_neon = nn.get_outputs(example_ram_Neon)

In [25]:
# A,B,down,left,right,select,start,up
input_transform_keys = ["A", "B", "down", "left", "right", "select", "start",
                        "up"]
wheighted_random = False
threshold = np.zeros(8, dtype=np.float32) + 0.5
def neon_input_to_emu(neon_input: np.ndarray):
    # not schour if wheighted random is a good joice
    if wheighted_random:
        input_booleans = neon_input > np.random.uniform(0, 1, 8)
    else:
        input_booleans = neon_input > threshold
    numpy_values = [r[0] for r in input_booleans.T]
    return dict(zip(input_transform_keys, numpy_values))
neon_input_to_emu(example_input_neon)

{'A': False,
 'B': False,
 'down': False,
 'left': False,
 'right': False,
 'select': False,
 'start': False,
 'up': False}

In [48]:
def run_simulation(emu):
    # sanity hack, to make should nothing is left in the buffer
    # prim_soc.recv(2**10)
    emu.load_slot(10)
    emu.speed_mode("normal")
    emu.step()
    ram = emu.get_ram()
    for i in range(60 * 60):
        ram_iter= ArrayIterator(np.frombuffer(ram,dtype=np.uint8).reshape((1,2048)),make_onehot=False)
        neon_input = nn.get_outputs(ram_iter)
        nex_input = neon_input_to_emu(neon_input)
        ram = emu.full_step(nex_input)

In [55]:
emu.unpause()
run_simulation(emu)

ValueError: total size of new array must be unchanged

In [35]:
for i in range(2048):
    prim_soc.recv(1)

KeyboardInterrupt: 